In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

Primero los datos estandarizados y codificados:

In [2]:
df=pd.read_csv('data/ocasionales_est_encod.csv',index_col=0)
df.head(3)

,registro,clima,temperatura,sens_termica,humedad,viento,ocasionales,fecha_nueva,festividad,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,ocasionales_BOX,año_map,estacion_map
0,1,2,-0.828948,-0.678635,1.295131,-0.376545,331,2018-01-01,New Year's Day,1,1,0,0,16.397820,0,0
1,2,2,-0.722312,-0.740034,0.507407,0.882110,131,2018-01-02,NaN,0,1,1,1,11.472437,0,0
2,3,1,-1.642047,-1.746977,-1.354487,0.861132,120,2018-01-03,NaN,0,1,2,1,11.076035,0,0


In [3]:
df.drop(['registro','fecha_nueva','festividad', 'ocasionales_BOX'], axis=1,inplace=True)
df.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'ocasionales', 'festividad_1', 'mes_bueno', 'dia_semana_nuevo',
       'no_laboral_nuevo', 'año_map', 'estacion_map'],
      dtype='object')

In [4]:
X=df.drop(['ocasionales'],axis=1)
y=df['ocasionales']

In [5]:
# y dividir nuestros datos en train y test para poder evaluar la bondad de nuestro modelo

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [6]:
max_features = np.sqrt(len(x_train.columns))
max_features

3.3166247903554

In [7]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param = {"max_depth": [2,4,6], "max_features": [1,2,3], "min_samples_split": [10, 150, 300], "min_samples_leaf": [10,150,300]}

In [8]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [9]:
# ajustamos el modelo que acabamos de definir en el GridSearch
# 🚨 Esta celda puede tardar en ejecutarse

gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 150, 300],
                         'min_samples_split': [10, 150, 300]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [10]:
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=6, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [11]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [12]:

def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [14]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

rf_ocasionales_estand_codif = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest ocasionales  estand-codif")
rf_ocasionales_estand_codif 

,MAE,MSE,RMSE,R2,set,modelo
0,275.200775,153456.750198,391.735562,0.680581,test,Random Forest ocasionales estand-codif
1,264.737005,141750.947832,376.498271,0.696923,train,Random Forest ocasionales estand-codif


## Ahora para los datos sin estandarizar:

In [15]:
df2 = pd.read_csv('data/ocasionales_encod.csv', index_col = 0)

In [16]:
df2.drop(['registro','fecha_nueva','festividad', 'ocasionales_BOX'], axis=1,inplace=True)
df2.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'ocasionales', 'festividad_1', 'mes_bueno', 'dia_semana_nuevo',
       'no_laboral_nuevo', 'año_map', 'estacion_map'],
      dtype='object')

In [18]:
X2=df.drop(['ocasionales'],axis=1)
y2=df['ocasionales']

In [19]:
# y dividir nuestros datos en train y test para poder evaluar la bondad de nuestro modelo

x2_train, x2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2, random_state = 42)

In [23]:
max_features2 = np.sqrt(len(x2_train.columns))
max_features2

3.3166247903554

In [36]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param2 = {"max_depth": [3,7,10], "max_features": [1,2,3], "min_samples_split": [10, 150, 300], "min_samples_leaf": [10,150,300]}

In [37]:
gs_rf2 = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param2, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [38]:
gs_rf2.fit(x2_train, y2_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [3, 7, 10], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 150, 300],
                         'min_samples_split': [10, 150, 300]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [33]:
bosque2 = gs_rf2.best_estimator_
bosque2

RandomForestRegressor(max_depth=7, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [34]:
y2_pred_test_rf = bosque2.predict(x2_test)
y2_pred_train_rf = bosque2.predict(x2_train)

In [35]:
rf_ocasionales_codif = metricas(y2_test, y2_train, y2_pred_test_rf, y2_pred_train_rf, "Random Forest ocasionales codificado")
rf_ocasionales_codif 

,MAE,MSE,RMSE,R2,set,modelo
0,261.779694,143253.225941,378.488079,0.701819,test,Random Forest ocasionales codificado
1,252.002262,130212.207520,360.849286,0.721594,train,Random Forest ocasionales codificado
